# ONNX to TF

In [1]:
import onnx

from onnx_tf.backend import prepare

onnx_model_path = "yolov7-tiny.onnx"
tf_model_path = "yolov7-tiny-tf"

onnx_model = onnx.load(onnx_model_path)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(tf_model_path)  # export the model

2023-04-06 22:11:40.419708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-06 22:11:40.419762: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/hanyong/miniconda3/envs/yolo7/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's

BackendIsNotSupposedToImplementIt: in user code:

    File "/home/hanyong/miniconda3/envs/yolo7/lib/python3.10/site-packages/onnx_tf/backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "/home/hanyong/miniconda3/envs/yolo7/lib/python3.10/site-packages/onnx_tf/backend.py", line 349, in _onnx_node_to_tensorflow_op  *
        raise BackendIsNotSupposedToImplementIt("{} is not implemented.".format(

    BackendIsNotSupposedToImplementIt: EfficientNMS_TRT is not implemented.


## TF Model Inference

In [ ]:
import tensorflow as tf

batch_size, width, height, channels = 1, 320, 320, 3
input_shape = (batch_size, width, height, channels)

model = tf.saved_model.load(tf_model_path)
model.trainable = False

input_tensor = tf.random.uniform(input_shape, 0, 255)
out = model(images=input_tensor)
for k in out:
    print(k, out[k].shape)

2023-04-05 15:01:28.220579: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: StatefulPartitionedCall/assert_equal_5/Assert/AssertGuard/branch_executed/_212


442 (1, 100, 4)
417 (100, 1)
438 (1,)
output (1, 100)


# TF to TFLite

In [ ]:
import tensorflow as tf
import os

os.environ['LD_LIBRARY_PATH'] = ' ${CONDA_PREFIX}/lib:${LD_LIBRARY_PATH}'

tflite_model_path = 'yolov7-tiny.tflite'
# tf_model_path = "yolov7-tiny-tf"

# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
# model = tf.keras.models.load_model(tf_model_path, compile=False)
# func = tf.function(model).get_concrete_function(
#     images=tf.TensorSpec(input_shape, tf.float32)
# )
# converter = tf.lite.TFLiteConverter.from_concrete_functions([func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

2023-04-05 15:05:29.061534: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-04-05 15:05:29.061606: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-04-05 15:05:29.061868: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: yolov7-tiny-tf
2023-04-05 15:05:29.079767: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-04-05 15:05:29.079818: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: yolov7-tiny-tf
2023-04-05 15:05:29.112069: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-04-05 15:05:29.332275: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: yolov7-tiny-tf
2023-04-05 15:05:29.541171: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 479304 micros

## Set model metadata

In [ ]:
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb

""" ... """
"""Creates the metadata for an image classifier."""

LABLE_FILE = os.path.basename('label.txt')
populator = _metadata.MetadataPopulator.with_model_file(tflite_model_path)
populator.load_associated_files([LABLE_FILE])

# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = tflite_model_path.split('.')[0]

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.name = "image"
input_meta.description = ("image")
input_meta.content = _metadata_fb.ContentT()
input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (
    _metadata_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.ImageProperties)
input_normalization = _metadata_fb.ProcessUnitT()
input_normalization.optionsType = (
    _metadata_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = _metadata_fb.NormalizationOptionsT()
input_normalization.options.mean = [127.5]
input_normalization.options.std = [127.5]
input_meta.processUnits = [input_normalization]
input_stats = _metadata_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_location_meta = _metadata_fb.TensorMetadataT()
output_location_meta.name = "location"
output_location_meta.description = "The locations of the detected boxes."
output_location_meta.content = _metadata_fb.ContentT()
output_location_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.BoundingBoxProperties)
output_location_meta.content.contentProperties = (
    _metadata_fb.BoundingBoxPropertiesT())
output_location_meta.content.contentProperties.index = [1, 0, 3, 2]
output_location_meta.content.contentProperties.type = (
    _metadata_fb.BoundingBoxType.BOUNDARIES)
output_location_meta.content.contentProperties.coordinateType = (
    _metadata_fb.CoordinateType.RATIO)
output_location_meta.content.range = _metadata_fb.ValueRangeT()
output_location_meta.content.range.min = 2
output_location_meta.content.range.max = 2

output_class_meta = _metadata_fb.TensorMetadataT()
output_class_meta.name = "category"
output_class_meta.description = "The categories of the detected boxes."
output_class_meta.content = _metadata_fb.ContentT()
output_class_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_class_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_class_meta.content.range = _metadata_fb.ValueRangeT()
output_class_meta.content.range.min = 2
output_class_meta.content.range.max = 2
label_file = _metadata_fb.AssociatedFileT()
label_file.name = LABLE_FILE
label_file.description = "Label of objects that this model can recognize."
label_file.type = _metadata_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_class_meta.associatedFiles = [label_file]

output_score_meta = _metadata_fb.TensorMetadataT()
output_score_meta.name = "score"
output_score_meta.description = "The scores of the detected boxes."
output_score_meta.content = _metadata_fb.ContentT()
output_score_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_score_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())
output_score_meta.content.range = _metadata_fb.ValueRangeT()
output_score_meta.content.range.min = 2
output_score_meta.content.range.max = 2

output_number_meta = _metadata_fb.TensorMetadataT()
output_number_meta.name = "number of detections"
output_number_meta.description = "The number of the detected boxes."
output_number_meta.content = _metadata_fb.ContentT()
output_number_meta.content.contentPropertiesType = (
    _metadata_fb.ContentProperties.FeatureProperties)
output_number_meta.content.contentProperties = (
    _metadata_fb.FeaturePropertiesT())

# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_location_meta, output_score_meta, output_class_meta, output_number_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(
    model_meta.Pack(b),
    _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

populator.load_metadata_buffer(metadata_buf)
populator.populate()

## TFLite model inference

In [ ]:
import numpy as np
import tensorflow as tf

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# get_tensor() returns a copy of the tensor data
# use tensor() in order to get a pointer to the tensor
for i in range(4):
    print(output_details[i]['index'], interpreter.get_tensor(output_details[i]['index']).shape)

522 (1, 100, 4)
548 (1,)
544 (1, 100)
535 (100, 1)
